In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime
from os import environ
from services.zayev.service_layer.agent import PPOAgent as ZPPO
from matplotlib.dates import relativedelta
import numpy as np


2023-11-22 01:08:25.282801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<Figure size 1800x900 with 0 Axes>

In [2]:

starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 2000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5
} 



zayev = ZPPO(env_config)

/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [3]:
print(zayev.Actor.Actor.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 stock_observation_input (I  [(None, 5, 5)]               0         []                            
 nputLayer)                                                                                       
                                                                                                  
 commodity_observation_inpu  [(None, 6, 5)]               0         []                            
 t (InputLayer)                                                                                   
                                                                                                  
 volume_observation_input (  [(None, 5, 5)]               0         []                            
 InputLayer)                                                                                  

In [5]:
zayev.load()
# zayev.save()

In [4]:
np.random.seed(5)
zayev.run_batch()
# zayev.run_multiprocesses(num_worker=4)

episode: 16/20, score: 3.566792441578194, average: 1.16 
episode: 17/20, score: 0.467221015118642, average: 1.12 
episode: 18/20, score: 2.280111762199533, average: 1.19 
episode: 19/20, score: 0.8909678402580693, average: 1.17 
episode: 20/20, score: 4.985705603523998, average: 1.36 


In [4]:
a_stck_condition = zayev.env.stcks_buffer_df['captured_at'] == datetime.fromisoformat("1995-01-22T16:00:00+00:00")
a_filtered_stck_df = zayev.env.stcks_buffer_df.loc[a_stck_condition]
# print(a_stck_condition)
print(a_filtered_stck_df)
print(zayev.env.stcks_buffer_df["captured_at"][1000])
print((zayev.env.the_current_time_step.replace(tzinfo=None) - datetime(year=1995, month=1, day=1, hour=10)).total_seconds()/(3600*2.5))

       id               captured_at  price_snapshot    change        volume  \
915  5551 1995-01-22 16:00:00+00:00       98.702231  4.970577   4007.062237   
916  5552 1995-01-22 16:00:00+00:00       25.037440 -0.466621   8792.804488   
917  5553 1995-01-22 16:00:00+00:00       38.328534  0.225369  15397.836814   
918  5554 1995-01-22 16:00:00+00:00      121.984370 -3.442836  49533.062425   
919  5555 1995-01-22 16:00:00+00:00       28.793264  0.522759   9301.270765   

         bid_vol   bid_price    offer_vol  offer_price  stock_id  index  
915   526.865920   98.889961   523.697493    99.003572        13      3  
916   910.216679   25.054586   883.722837    25.096208        15      5  
917  1624.442924   38.331448  3437.426342    38.345048        12      2  
918   649.724116  122.001654   968.868751   122.051076        11      1  
919   198.951127   28.799958   220.297125    28.839932        14      4  
1995-01-02 01:00:00+00:00
170.0


In [5]:
# print(zayev.Actor.Actor)
# [-1.         -0.99999756  0.9822062  -1.         -0.99031776]
# zayev.load()
zayev.test()

In [6]:
# print(zayev.previous_state)
print(zayev.previous_actn)
# print((zayev.env.the_current_time_step.replace(tzinfo=None) - datetime(year=1995, month=1, day=1, hour=10)).total_seconds()/3600)
# print(zayev.latest_state)
asum = 0
for act in zayev.previous_actn:
    asum = asum + act
print(asum)
# [ 0.7059819  -0.42333204 -0.3999321   0.29401806 -0.01280397 -0.16393185]

[-0.22598502  0.00877554  0.20621498 -0.47725093  0.78500944 -0.29676402]
-7.450580596923828e-09


In [7]:
ztt = zayev.latest_state
zt_env = zayev.env

total_potfolio = zayev.env.wallet_state
for ind in range(5):
    data_index = ind+1
    total_potfolio = total_potfolio + zt_env.stock_data[data_index]["price_snapshot"]*zt_env.shares_data[data_index]
print(total_potfolio)
print(zayev.env.the_current_time_step - relativedelta(hours=100))

147542.63323069585
1995-01-25 21:30:00+00:00


In [8]:
tm = starting_time_step + relativedelta(hours=25)
print(tm)

1995-01-02 11:00:00


In [6]:
zayev.latest_state

[array([[[  86.98892013, 1780.88718796,  806.78320262,   86.99418474,
           815.32319724,   87.01223627,   86.22798652,  231.66143027],
         [  24.55947431, 5311.76025676, 2294.7676268 ,   24.5622656 ,
          2544.55969405,   24.56600397,   24.61056939,  795.63826776],
         [ 139.2253307 , 1086.44600003,  505.13989644,  139.25715658,
           561.13738873,  139.33339698,  144.20254547,  126.31578947],
         [  38.81905216,  614.39851888,  328.47856092,   38.82553138,
           308.16662239,   38.84516044,   38.48886214,  533.33333333],
         [  21.18280565, 1786.27803173,  976.56194102,   21.18713267,
           841.46566394,   21.19766482,   21.08023049,  937.5       ]]]),
 array([[[1.71481016e+02],
         [3.49922248e+00],
         [4.32156117e+02],
         [7.54711667e+03],
         [2.72181363e+04],
         [3.93169616e+03]]]),
 array([[0]])]

In [8]:
actp = np.array([1,2,3])*1000
print(actp)

[1000 2000 3000]


In [13]:
log_std = .5 * np.ones(5, dtype=np.float32)
std = np.exp(log_std)

print(log_std)
print(std)

[0.5 0.5 0.5 0.5 0.5]
[1.6487212 1.6487212 1.6487212 1.6487212 1.6487212]


In [6]:
# print(zayev.env.the_current_time_step)

1995-01-16 20:00:00+00:00


In [22]:

def check(s):

    brac_map = {
        "{": "}",
        "(": ")",
        "[": "]"
    }

    sub_brac_map = {
        "}": "{",
        ")": "(",
        "]": "["
    }

    mystr = [x for x in s]
    new_dict = {
        "{": [],
        "[": [],
        "(": []
    }
    for a  in mystr:
        if a in brac_map:
            new_dict[a].append(a)
        elif a in sub_brac_map:
            new_dict[sub_brac_map[a]].append(a)
    
    for key, bracs in new_dict.items():
        if bracs[0] not in brac_map or bracs[-1] not in sub_brac_map:
            return False
        if len(bracs)%2 != 0:
            return False
        openin = 0
        clsing = 0
        for val in bracs:
            if val in brac_map:
                openin = openin +1
            elif val in sub_brac_map:
                clsing = clsing+1
        if openin-clsing!= 0:
            return False
    return True
        
print(check('()[][[({)]]{}'))


False


In [56]:
def chck_ways(num_of_rocks, num_ways):
    if num_of_rocks <=1:
        return 1
    else:
        new_ways = 0
        new_ways = new_ways + chck_ways(num_of_rocks-1, num_ways)
        new_ways = new_ways + chck_ways(num_of_rocks-2, num_ways)
        num_ways = num_ways+new_ways
        return num_ways
    

print(chck_ways(6,0))

13


In [57]:
import os
os.urandom(24)

b'\x97\x99\xf3\xfe\x15\x89\xa58\xa3\xdcV\x0b\x94\xbb\xb6UIR@\xf7H\x85.-'